In [8]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import IPython
import librosa

import scripts.afilter as afilter
import scripts.sigutil as util

from sklearn.metrics import mean_squared_error

In [ ]:
importlib.reload(afilter)
importlib.reload(util)

### Utils

In [5]:
def plot_adaptive(i, sig):
    if (i - 2*fs) % (8*fs) == 0 and i > 10*fs:
        plt.title(f'Room impulse response at {i/fs} s')
        plt.plot(np.arange(len(sig))/fs, sig)
        plt.xlabel('Time [s]')
        plt.ylabel('Amplitude')
        plt.grid()
        plt.show()

def plot_results(d, e, s, title):
    plt.title(title)
    plt.plot(np.arange(len(d))/fs, d, color='blue', label='d[n]', alpha=0.7)
    plt.plot(np.arange(len(e))/fs, e, color='red', label='e_ad[n]', alpha=0.7)
    plt.plot(np.arange(len(s))/fs, s, color='green', label='s[n], clean signal', alpha=0.5)
    plt.grid()
    plt.legend()

### MSE
We use the music + noise model, as we have access to the clean recording.

In [9]:
fs = 8000 # Hz

cleanSig, _ = librosa.load('data/3_CleanGuitar.wav', sr=fs) # Downsample to fs 
noise, _ = librosa.load('data/talkingNoise.wav', sr=fs)

length = min(len(cleanSig), len(noise))
x = noise[:length]*.08
d = cleanSig[:length] + x
cleanSig = cleanSig[:length]

#### LMS

In [ ]:
mu = 0.03
K = 200  # number of taps
N_it = 5 # number of iterations

f_ad_lms, e_lms = afilter.adaptive_filter(x, d, K, N_it, 
                                    algoType='LMS', mu=mu, callback=None)

In [ ]:
plot_results(d, e_lms, cleanSig, title='rls')

In [ ]:
mse_rls = mean_squared_error(y_true=cleanSig, y_pred=e_lms)

#### NLMS

In [ ]:
mu = .1  # needs to be between 0 and 2
K = 200  # number of taps
N_it = 10  # number of iterations

f_ad_nlms, e_nlms = afilter.adaptive_filter(x, d, K, N_it, 
                                    algoType='NLMS', mu=mu, callback=None)

In [ ]:
plot_results(d, e_nlms, cleanSig, title='rls')
mse_rls = mean_squared_error(y_true=cleanSig, y_pred=e_nlms)

#### RLS

In [10]:
delta = 0.01 # Regularization. Smaller delta => stronger filter.
lambda_ = 0.9 # Forgetting factor.
K = 200  # number of taps
N_it = 1  # number of iterations

f_ad_rls, e_rls = afilter.adaptive_filter(x, d, K, N_it, 
                                            algoType='RLS', lambda_=lambda_, delta=delta, callback=None)

In [ ]:
plot_results(d, e_rls, cleanSig, title='rls')

In [14]:
mse_rls = mean_squared_error(y_true=cleanSig, y_pred=e_rls)

### Speech metrics